In [ ]:
%cd /content/drive/MyDrive/FM

/content/drive/MyDrive/FM


In [ ]:
def generate_prism_model(through_vehicles, non_through_vehicles):
    # Define constants and global variables
    maxQueueSize = max(through_vehicles, non_through_vehicles) + 1
    total_vehicles = through_vehicles + non_through_vehicles
    dummy_vehicle_var = f"v{total_vehicles}"

    model = f"""dtmc

// Define constants
const int maxQueueSize = {maxQueueSize};
const int congestionThreshold = 3;

// Define vehicle statuses as integer values
const int runningStatus = 0;
const int approachingStatus = 1;
const int stoppedStatus = 2;
const int crossingStatus = 3;
const int crossedStatus = 4;
const int unspaceStatus = 5;
const int spaceStatus = 6;
const int yieldStatus = 7;

// Global variables for vehicle queues and status flags
global throughQueue : [0..maxQueueSize] init 1;
global nonThroughQueue : [0..maxQueueSize] init 0;
global mode : [0..1] init 0; // 0 = prioritized, 1 = fair
global isCrossing : bool init false;
global turn : [0..1] init 0; // 0 = through, 1 = nonThrough
global dPos : [0..{through_vehicles}] init 0; // Dummy vehicle position
global CountPos : [0..{maxQueueSize}] init 0; // Number of Through lane cars crossed
global dummyFirst : bool init true; // Flag to ensure dummy vehicle moves first

// Formula for the total number of vehicles in queues
formula ucvs = throughQueue + nonThroughQueue;
"""

    # Generate first through lane vehicle module
    model += f"""
// First vehicle on through lanes
module Vehicle0

    v0 : [0..7] init approachingStatus; // Set it to approaching to check for termination label, since we need at least 1 vh in queue

    // Approaching transition
    // [] !dummyFirst & v0 = runningStatus & throughQueue < maxQueueSize -> (v0' = approachingStatus) & (throughQueue' = throughQueue + 1);

    // Immediate crossing if in prioritized mode and no vehicle is crossing
    [] !dummyFirst & v0 = approachingStatus & (mode = 0 | nonThroughQueue = 0) & !isCrossing -> (v0' = crossingStatus) & (isCrossing' = true);
    [] !dummyFirst & v0 = approachingStatus & ( mode = 1 & turn = 0) & !isCrossing -> (v0' = crossingStatus) & (isCrossing' = true) & (turn' = 1);

    // Stopping transition
    [] v0 = approachingStatus & isCrossing -> (v0' = stoppedStatus);

    // Entering transition
    [] v0 = stoppedStatus & mode = 0 & !isCrossing -> (v0' = crossingStatus) & (isCrossing' = true);
    [] v0 = stoppedStatus & mode = 1 & turn = 0 & !isCrossing -> (v0' = crossingStatus) & (turn' = 1) & (isCrossing' = true);

    // Exiting transition
    [] v0 = crossingStatus & throughQueue > 0 & CountPos < {maxQueueSize} -> (v0' = crossedStatus) & (isCrossing' = false) & (throughQueue' = throughQueue - 1) & (CountPos' = CountPos + 1);

endmodule
"""

    # Generate remaining through lane vehicle modules
    for i in range(1, through_vehicles):
        model += f"""
module Vehicle{i}

    v{i} : [0..7] init runningStatus;

    // Approaching transition
    [] !dummyFirst & v{i} = runningStatus & throughQueue < maxQueueSize -> (v{i}' = approachingStatus) & (throughQueue' = throughQueue + 1);

    // Immediate crossing if in prioritized mode and no vehicle is crossing
    [] v{i} = approachingStatus & (mode = 0 | nonThroughQueue = 0) & !isCrossing & v{i-1} = crossedStatus -> (v{i}' = crossingStatus) & (isCrossing' = true);
    [] v{i} = approachingStatus & ( mode = 1 & turn = 0) & !isCrossing & v{i-1} = crossedStatus -> (v{i}' = crossingStatus) & (isCrossing' = true) & (turn' = 1);

    // Stopping transition
    [] v{i} = approachingStatus & ((v{i-1} = stoppedStatus | v{i-1} = crossingStatus) | (isCrossing & v{i-1} = crossedStatus)) -> (v{i}' = stoppedStatus);

    // Entering transition
    [] v{i} = stoppedStatus & mode = 0 & !isCrossing & v{i-1} = crossedStatus -> (v{i}' = crossingStatus) & (isCrossing' = true);
    [] v{i} = stoppedStatus & mode = 1 & turn = 0 & !isCrossing & v{i-1} = crossedStatus -> (v{i}' = crossingStatus) & (turn' = 1) & (isCrossing' = true);

    // Exiting transition
    [] v{i} = crossingStatus & throughQueue > 0 & CountPos < {maxQueueSize} -> (v{i}' = crossedStatus) & (isCrossing' = false) & (throughQueue' = throughQueue - 1) & (CountPos' = CountPos + 1);

endmodule
"""

    # Generate first non-through lane vehicle module
    model += f"""
// First vehicle on non-through lanes
module Vehicle{through_vehicles}

    v{through_vehicles} : [0..7] init runningStatus;

    // Approaching transition
    [] !dummyFirst & v{through_vehicles} = runningStatus & nonThroughQueue < maxQueueSize -> (v{through_vehicles}' = approachingStatus) & (nonThroughQueue' = nonThroughQueue + 1);

    // Immediate crossing if in prioritized mode and no vehicle is crossing
    [] v{through_vehicles} = approachingStatus & ((mode = 0 & CountPos+1 = dPos) | throughQueue = 0) & !isCrossing -> (v{through_vehicles}' = crossingStatus) & (isCrossing' = true);
    [] v{through_vehicles} = approachingStatus & ( mode = 1 & turn = 1) & !isCrossing -> (v{through_vehicles}' = crossingStatus) & (isCrossing' = true) & (turn' = 0);

    // Stopping transition
    [stopped] v{through_vehicles} = approachingStatus & isCrossing -> (v{through_vehicles}' = stoppedStatus);

    // Entering transition
    [] v{through_vehicles} = stoppedStatus & mode = 0 & !isCrossing -> (v{through_vehicles}' = crossingStatus) & (isCrossing' = true);
    [] v{through_vehicles} = stoppedStatus & ( mode = 1 & turn = 1 ) & !isCrossing -> (v{through_vehicles}' = crossingStatus) & (turn' = 0) & (isCrossing' = true);

    // Exiting transition
    [] v{through_vehicles} = crossingStatus & nonThroughQueue > 0 -> (v{through_vehicles}' = crossedStatus) & (isCrossing' = false) & (nonThroughQueue'=nonThroughQueue-1);

endmodule
"""

    # Generate intermediate non-through lane vehicle modules
    for i in range(1, non_through_vehicles - 1):
        idx = through_vehicles + i
        model += f"""
module Vehicle{idx}

    v{idx} : [0..7] init runningStatus;

    // Approaching transition
    [] !dummyFirst & v{idx} = runningStatus & nonThroughQueue < maxQueueSize -> (v{idx}' = approachingStatus) & (nonThroughQueue' = nonThroughQueue + 1);

    // Immediate crossing if in prioritized mode and no vehicle is crossing
    [] v{idx} = approachingStatus & ((mode = 0 & CountPos+1 = dPos) | throughQueue = 0) & !isCrossing & v{idx-1} = crossedStatus -> (v{idx}' = crossingStatus) & (isCrossing' = true);
    [] v{idx} = approachingStatus & ( mode = 1 & turn = 1) & !isCrossing & v{idx-1} = crossedStatus -> (v{idx}' = crossingStatus) & (isCrossing' = true) & (turn' = 0);

    // Stopping transition
    [] v{idx} = approachingStatus & ((isCrossing & v{idx-1} = crossedStatus) | (v{idx-1} = stoppedStatus | v{idx-1} = crossingStatus)) -> (v{idx}' = stoppedStatus);

    // Entering transition
    [] v{idx} = stoppedStatus & mode = 0 & !isCrossing & v{idx-1} = crossedStatus -> (v{idx}' = crossingStatus) & (isCrossing' = true);
    [] v{idx} = stoppedStatus & ( mode = 1 & turn = 1 ) & !isCrossing & v{idx-1} = crossedStatus -> (v{idx}' = crossingStatus) & (turn' = 0) & (isCrossing' = true);

    // Exiting transition
    [] v{idx} = crossingStatus & nonThroughQueue > 0 -> (v{idx}' = crossedStatus) & (isCrossing' = false) & (nonThroughQueue'=nonThroughQueue-1);

endmodule
"""

    # Generate last non-through lane vehicle module
    if non_through_vehicles != 1:
        idx = through_vehicles + non_through_vehicles - 1
        model += f"""
// Last vehicle on non-through lanes
module Vehicle{idx}

    v{idx} : [0..7] init runningStatus;

    // Approaching transition
    [] !dummyFirst & v{idx} = runningStatus & nonThroughQueue < maxQueueSize -> (v{idx}' = approachingStatus) & (nonThroughQueue' = nonThroughQueue + 1);

    // Immediate crossing if in prioritized mode and no vehicle is crossing
    [] v{idx} = approachingStatus & ((mode = 0 & CountPos+1 = dPos) | throughQueue = 0) & !isCrossing & v{idx-1} = crossedStatus -> (v{idx}' = crossingStatus) & (isCrossing' = true);
    [] v{idx} = approachingStatus & ( mode = 1 & turn = 1) & !isCrossing & v{idx-1} = crossedStatus -> (v{idx}' = crossingStatus) & (isCrossing' = true) & (turn' = 0);

    // Stopping transition
    [stopped2] v{idx} = approachingStatus & ((isCrossing & v{idx-1} = crossedStatus) | (v{idx-1} = stoppedStatus | v{idx-1} = crossingStatus)) -> (v{idx}' = stoppedStatus);

    // Entering transition
    [] v{idx} = stoppedStatus & mode = 0 & !isCrossing & v{idx-1} = crossedStatus -> (v{idx}' = crossingStatus) & (isCrossing' = true);
    [] v{idx} = stoppedStatus & ( mode = 1 & turn = 1 ) & !isCrossing & v{idx-1} = crossedStatus -> (v{idx}' = crossingStatus) & (turn' = 0) & (isCrossing' = true);

    // Exiting transition
    [] v{idx} = crossingStatus & nonThroughQueue > 0 -> (v{idx}' = crossedStatus) & (isCrossing' = false) & (nonThroughQueue'=nonThroughQueue-1);

endmodule
"""

    # Generate dummy vehicle module with dynamic positions
    dummy_vehicle_transitions = "\n".join(
        [f"""    [] {dummy_vehicle_var} = spaceStatus & dPos = {i+1} & (v{i} = crossingStatus | v{i} = stoppedStatus | v{i} = crossedStatus) & throughQueue > 0 & CountPos < {maxQueueSize}  ->
            ({dummy_vehicle_var}' = yieldStatus) & (throughQueue' = throughQueue - 1) & (CountPos' = CountPos + 1);""" for i in range(through_vehicles)]
    )

    model += f"""
// Dummy vehicle module
module DummyVehicle

    {dummy_vehicle_var} : [0..7] init unspaceStatus;

    // Randomly choose a position in the through queue + 1 for dummy vehicle
    [] dummyFirst & {dummy_vehicle_var} = unspaceStatus & throughQueue < maxQueueSize -> {" + ".join([f"1/{through_vehicles} : (dPos' = {i+1}) & ({dummy_vehicle_var}' = spaceStatus) & (throughQueue' = throughQueue + 1) & (dummyFirst' = false)" for i in range(through_vehicles)])};

{dummy_vehicle_transitions}

endmodule

// Control module for mode based on congestion
module Control

    [] throughQueue >= congestionThreshold -> (mode' = 1);
    [] throughQueue < congestionThreshold -> (mode' = 0);
    [] mode = 1 & turn = 1 & nonThroughQueue = 0 -> (turn' = 0);
    [] mode = 1 & turn = 0 & throughQueue = 0 -> (turn' = 1);

endmodule

// Rewards structure for time spent in waiting states (approaching and stopped)
rewards "waiting_time_first" //waiting time for first non-through lane vehicle
    [stopped] true : 1;
endrewards
"""
    if non_through_vehicles != 1:
        model += """
rewards "waiting_time_last" //waiting time for last non-through lane vehicle
    [stopped2] true : 1;
endrewards
"""
    model += f"""
// Property Specification
label "simultaneous_crossing" = {" | ".join([f"(v{i} = crossingStatus & v{j} = crossingStatus)" for i in range(through_vehicles + non_through_vehicles) for j in range(i + 1, through_vehicles + non_through_vehicles)])};

label "termination" = (ucvs = 0);
"""

    return model


In [ ]:
def generate_properties(through_vehicles, non_through_vehicles):
    # Base properties
    properties = [
        'P=? [ F "simultaneous_crossing" ]',
        'P=? [ F "termination" ]',
    ]

    if non_through_vehicles!=1:
      properties.append('R{"waiting_time_last"}=? [ C<=50 ]')

    properties.append('P=? [ F (throughQueue=maxQueueSize) ]')


    # Extended 5th property
    crossed_through = " & ".join([f"v{i}=crossedStatus" for i in range(through_vehicles)])
    not_crossed_non_through = " & ".join([f"v{through_vehicles+i}!=crossedStatus" for i in range(non_through_vehicles)])
    properties.append(f'P=? [ F<=50 ({crossed_through}&({not_crossed_non_through})) ]')
    properties.append('R{"waiting_time_first"}=? [ C<=50 ]')

    return "\n".join(properties)

In [ ]:
for through_vehicles in range(1, 9):
  for non_through_vehicles in range(1, 9):
    model = generate_prism_model(through_vehicles, non_through_vehicles)
    filename = f"prism_model_{through_vehicles}x{non_through_vehicles}.prism"
    with open(filename, 'w') as f:
        f.write(model)

    properties = generate_properties(through_vehicles, non_through_vehicles)
    properties_filename = f"prism_properties_{through_vehicles}x{non_through_vehicles}.props"
    with open(properties_filename, 'w') as f:
        f.write(properties)

    print(f"Generated model and properties for {through_vehicles} through vehicles and {non_through_vehicles} non-through vehicles")


Generated model and properties for 1 through vehicles and 1 non-through vehicles
Generated model and properties for 1 through vehicles and 2 non-through vehicles
Generated model and properties for 1 through vehicles and 3 non-through vehicles
Generated model and properties for 1 through vehicles and 4 non-through vehicles
Generated model and properties for 1 through vehicles and 5 non-through vehicles
Generated model and properties for 1 through vehicles and 6 non-through vehicles
Generated model and properties for 1 through vehicles and 7 non-through vehicles
Generated model and properties for 1 through vehicles and 8 non-through vehicles
Generated model and properties for 2 through vehicles and 1 non-through vehicles
Generated model and properties for 2 through vehicles and 2 non-through vehicles
Generated model and properties for 2 through vehicles and 3 non-through vehicles
Generated model and properties for 2 through vehicles and 4 non-through vehicles
Generated model and properti